In [193]:
import pandas as pd
import numpy as np

In [194]:
movies = pd.read_csv('../aplikacja_polecania/Movies/movies.csv')
ratings = pd.read_csv('../Aplikacja_polecania/Movies/ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


Krok 1 znaleźenie najlepiej pasujących sąsiadów

In [195]:
mean = ratings.groupby(['userId'],as_index = False, sort = False).mean().rename(columns={'rating':'rating_mean'})#wyliczana jest średnia całkowita ocen filmów danego użytkownika
Ratings2 = pd.merge(ratings,mean,on='userId',how='left',sort=False)
Ratings2['ratings_adjusted'] = Ratings2['rating']-Ratings2['rating_mean']
result = pd.DataFrame({'user':Ratings2['userId'], #żeby lepiej ten przykład zrozumieć całość przedstawiłem w formie tabeli gdzie indexy to uzytkownicy a columny to numery id filmów
                       'movies':Ratings2['movieId_x'],                        
                       'ratings':Ratings2['ratings_adjusted']})
result_pivot = result.pivot_table(index = 'user',columns='movies',values='ratings').fillna(0)
result_pivot

movies,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
user,,,,,,,,,,,,,,,,,,,,,
1,-0.366379,0.000000,-0.366379,0.0,0.0,-0.366379,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.363636,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,-1.157399,0.000000,0.000000,0.0,0.0,0.000000,-1.157399,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.213904,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,-0.634176,-1.134176,-1.134176,0.0,0.0,0.000000,0.000000,0.0,0.0,0.865824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [196]:
all_users = result_pivot.values
user_number1=all_users[0]#user o id 1
sum_user1 = np.sqrt(np.sum(user_number1**2))#potrzebne do obliczenia podobieństwa z wykorzystaniem wzrou na podobieństwo cosinusa
sum_user1
list_of_users_with_similarity_coefficient = []
for count,user_values in enumerate(all_users):#te obliczenia dużo szybsze niż zaproponowane w artykule korzystniej jest wykorzystać 
    licznik = np.sum(user_number1*user_values)#funkcje numpyowe jeżeli działamy na arrayach
    sum_user = np.sqrt(np.sum(user_values**2))
    coefficent = licznik/(sum_user1*sum_user)
    list_of_users_with_similarity_coefficient.append((result_pivot.index[count],coefficent))

list_of_users_with_similarity_coefficient.sort(key = lambda x:x[1],reverse=True)
list_of_users_with_similarity_coefficient[0:10]

C:\Users\shina\AppData\Local\Temp\ipykernel_8660\3074672182.py:9: RuntimeWarning: invalid value encountered in double_scalars
  coefficent = licznik/(sum_user1*sum_user)


[(1, 1.0000000000000002),
 (301, 0.12479906517911302),
 (597, 0.10263065929063651),
 (414, 0.10134803449460647),
 (477, 0.09921664240127363),
 (369, 0.09829454350405929),
 (206, 0.09685159012412195),
 (535, 0.09649292687545016),
 (590, 0.09519062577565124),
 (418, 0.0941526193713487)]

In [197]:
top10users = pd.DataFrame()
for user in list_of_users_with_similarity_coefficient[0:10]:
    top10users = top10users.append(result_pivot.loc[user[0]])
top10users['costheta'] = [user[1] for user in list_of_users_with_similarity_coefficient[0:10]]

all_values = top10users.values
top10users

C:\Users\shina\AppData\Local\Temp\ipykernel_8660\146575234.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top10users = top10users.append(result_pivot.loc[user[0]])
C:\Users\shina\AppData\Local\Temp\ipykernel_8660\146575234.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top10users = top10users.append(result_pivot.loc[user[0]])
C:\Users\shina\AppData\Local\Temp\ipykernel_8660\146575234.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top10users = top10users.append(result_pivot.loc[user[0]])
C:\Users\shina\AppData\Local\Temp\ipykernel_8660\146575234.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top10users = top10users.ap

movies,1,2,3,4,5,6,7,8,9,10,...,193567,193571,193573,193579,193581,193583,193585,193587,193609,costheta
1,-0.366379,0.000000,-0.366379,0.0,0.000000,-0.366379,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
301,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.679825,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.124799
597,0.022573,0.000000,0.000000,0.0,0.000000,-0.977427,-2.977427,0.000000,0.0,-0.977427,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.102631
414,0.608043,-0.391957,0.608043,0.0,-1.391957,-0.391957,-0.391957,-0.391957,0.0,-0.391957,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.101348
477,0.263333,0.263333,-0.736667,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.099217
369,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.098295
206,0.960000,0.000000,0.000000,0.0,0.000000,0.000000,-0.040000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.096852
535,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.096493
590,0.644231,-0.855769,-0.355769,0.0,-1.355769,0.144231,0.000000,0.000000,0.0,0.144231,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.095191
418,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094153


Krok 2 czyli przewidzenie oceny użytkownika

In [198]:
top10users_with_grade = top10users.copy()
top10users_with_grade.drop(columns='costheta',inplace = True)

In [199]:
denominator = sum(top10users.loc[:,'costheta'])
for idx,movie_rate in enumerate(top10users.loc[1,:]):
    if movie_rate== 0.0:
        top10users_with_grade.iloc[0,idx] = top10users.iloc[:,idx].dot(top10users.loc[:,'costheta'])/denominator


In [214]:

moviesId_top10 = (top10users_with_grade
                  .iloc[0,]
                  .sort_values(ascending = False)
                  .reset_index().drop(columns=1)
                  .loc[:,'movies'])[0:10]

pandas.core.series.Series

In [218]:
movies.loc[movies['movieId'].isin(moviesId_top10),:]

,movieId,title,genres
124,151,Rob Roy (1995),Action|Drama|Romance|War
701,919,"Wizard of Oz, The (1939)",Adventure|Children|Fantasy|Musical
705,923,Citizen Kane (1941),Drama|Mystery
720,940,"Adventures of Robin Hood, The (1938)",Action|Adventure|Romance
734,954,Mr. Smith Goes to Washington (1939),Drama
2216,2944,"Dirty Dozen, The (1967)",Action|Drama|War
2226,2959,Fight Club (1999),Action|Crime|Drama|Thriller
2250,2987,Who Framed Roger Rabbit? (1988),Adventure|Animation|Children|Comedy|Crime|Fant...
2254,2991,Live and Let Die (1973),Action|Adventure|Thriller
2256,2993,Thunderball (1965),Action|Adventure|Thriller
